In [14]:
# import PyTorch
import torch
# import PyTorch Neural Network module
import torch.nn as nn
#random
import pandas as pd

# Build Model

Các hàm Sigmoid Activation

In [15]:
def sigmoid(value): # hàm sigmoid activation
    sig = 1 / (1 + torch.exp(-value))
    return sig

In [16]:
def sigmoid_derivative(value): # đạo hàm của hàm sigmoid activation
    d = value * (1 - value)
    return d

Class về Foward Feed Neural Network

In [17]:
class FFNN(nn.Module):
    # initialization function
    def __init__(self):
        # init function of base class
        super(FFNN, self).__init__()

        # corresponding size of each layer
        self.inputlayer = 3 # number of perceptrons at input layer
        self.hiddenlayer = 12 # number of perceptrons at hidden layer
        self.outputlayer = 1 # number of perceptrons at output layer

        # random weights from a normal distribution
        self.W1 = torch.randn(self.inputlayer, self.hiddenlayer)   # 3 X 12 tensor
        self.W2 = torch.randn(self.hiddenlayer, self.hiddenlayer)  # 12 X 12 tensor
        self.W3 = torch.randn(self.hiddenlayer, self.outputlayer)  # 12 X 1 tensor

        self.z = None # processing result between input layer and 1st hidden layer
        self.a = None # sigmoid activation result of z

        self.z_activation = None # save sigmoid activation result of z
        self.z_activation_derivative = None # save the gradient descent sigmoid activation result of z


        self.z2 = None # processing result between 1s hidden layer and 2nd hidden layer
        self.a2 = None # sigmoid activation result of z2

        self.z3 = None # processing result between 1s hidden layer and 2nd hidden layer
        self.a3 = None # sigmoid activation result of z3

        self.w3_error = None # Error value between correct result and predicted result
        self.w3_delta = None # result of error * derivative of activation function of ai result (i = i-th layer) (i=3)

        self.w2_error = None # Error value between correct result and predicted result
        self.w2_delta = None # result of error * derivative of activation function of ai result (i = 2)

        self.w1_error = None # Error value between correct result and predicted result
        self.w1_delta = None # result of error * derivative of activation function of ai result (i = 1)

        # a = sigmoid(z); z = x.T * W       

    # activation function using sigmoid
    def activation(self, z):
        self.z_activation = sigmoid(z)
        return self.z_activation

    # derivative of activation function
    def activation_derivative(self, z):
        self.z_activation_derivative = sigmoid_derivative(z)
        return self.z_activation_derivative

    def forward(self, X):
        # multiply input X and weights W1 from input layer to 1st hidden layer
        self.z = torch.matmul(X, self.W1)
        self.a = self.activation(self.z)  # activation function

        # multiply current tensor and weights W2 from 1st hidden layer to 2nd hidden layer
        self.z2 = torch.matmul(self.a, self.W2)
        self.a2 = self.activation(self.z2)  # activation function

        # multiply current tensor and weights W3 from 2nd hidden layer to output layer
        self.z3 = torch.matmul(self.a2, self.W3)
        self.a3 = self.activation(self.z3)  # activation function
        return self.a3

    def backward(self, X, y, res, rate):
        # W3 adjustment rate
        self.w3_error = y - res  # error in output
        self.w3_delta = self.w3_error * self.activation_derivative(res)  # derivative of activation to error

        # W2 adjustment rate
        self.w2_error = torch.matmul(self.w3_delta, torch.t(self.W3))
        self.w2_delta = self.w2_error * self.activation_derivative(self.a2)

        # W1 adjustment rate
        self.w1_error = torch.matmul(self.w2_delta, torch.t(self.W2))
        self.w1_delta = self.w1_error * self.activation_derivative(self.a)

        # update weights from delta of error and learning rate
        self.W1 += torch.matmul(torch.t(X), self.w1_delta) * rate
        self.W2 += torch.matmul(torch.t(self.a2), self.w2_delta) * rate
        self.W3 += torch.matmul(torch.t(self.a3), self.w3_delta) * rate

    # training function with learning rate parameter
    def train(self, X, y, rate):
        # forward + backward pass for training
        result = self.forward(X)
        self.backward(X, y, result, rate)

    # predict function
    def predict(self, x_predict):
        print("Predict data based on trained weights: ")
        print("Input: \n" + str(x_predict))
        print("Output: \n" + str(self.forward(x_predict)))

Lưu trained model

In [18]:
def SaveModel(model, name):
   torch.save(model, name)

Load model

In [19]:
def LoadModel(name):
   model = torch.load(name)
   return model

# Chạy Model

Khởi tạo dữ liệu

In [20]:
x_train = pd.read_csv('data.csv', usecols=[0,1,2]) # Read first 3 variable columns in dataset
x_train = x_train.values.tolist() # Convert dataframe to list

y_train = pd.read_csv('data.csv', usecols=[3]) # Read result columns in dataset
y_train = y_train.values.tolist() # Convert dataframe to list

các dữ liệu về x_train, y_train

In [21]:
print(x_train)

[[44.5, 39.3, 45.1], [17.2, 45.9, 69.3], [151.5, 41.3, 58.5], [180.8, 10.8, 58.4], [8.7, 48.9, 75.0], [57.5, 32.8, 23.5], [120.2, 19.6, 11.6], [8.6, 2.1, 1.0], [199.8, 2.6, 21.2], [66.1, 5.8, 24.2], [214.7, 24.0, 4.0], [23.8, 35.1, 65.9], [97.5, 7.6, 7.2], [204.1, 32.9, 46.0], [195.4, 47.7, 52.9], [67.8, 36.6, 114.0], [281.4, 39.6, 55.8], [69.2, 20.5, 18.3], [147.3, 23.9, 19.1], [218.4, 27.7, 53.4], [237.4, 5.1, 23.5], [13.2, 15.9, 49.6], [228.3, 16.9, 26.2], [62.3, 12.6, 18.3], [262.9, 3.5, 19.5], [142.9, 29.3, 12.6], [240.1, 16.7, 22.9], [248.8, 27.1, 22.9], [70.6, 16.0, 40.8], [292.9, 28.3, 43.2], [112.9, 17.4, 38.6], [97.2, 1.5, 30.0], [265.6, 20.0, 0.3], [95.7, 1.4, 7.4], [290.7, 4.1, 8.5], [266.9, 43.8, 5.0], [74.7, 49.4, 45.7], [43.1, 26.7, 35.1], [228.0, 37.7, 32.0], [202.5, 22.3, 31.6], [177.0, 33.4, 38.7], [293.6, 27.7, 1.8], [206.9, 8.4, 26.4], [25.1, 25.7, 43.3], [175.1, 22.5, 31.5], [89.7, 9.9, 35.7], [239.9, 41.5, 18.5], [227.2, 15.8, 49.9], [66.9, 11.7, 36.8], [199.8, 3.

In [22]:
print(y_train)

[[10.4], [12.0], [16.5], [17.9], [7.2], [11.8], [13.2], [4.8], [15.6], [12.6], [17.4], [9.2], [13.7], [19.0], [22.4], [12.5], [24.4], [11.3], [14.6], [18.0], [17.5], [5.6], [20.5], [9.7], [17.0], [15.0], [20.9], [18.9], [10.5], [21.4], [11.9], [13.2], [17.4], [11.9], [17.8], [25.4], [14.7], [10.1], [21.5], [16.6], [17.1], [20.7], [17.9], [8.5], [16.1], [10.6], [23.2], [19.8], [9.7], [16.4], [10.7], [22.6], [21.2], [20.2], [23.7], [5.5], [13.2], [23.8], [18.4], [8.1], [24.2], [20.7], [14.0], [16.0], [11.3], [11.0], [13.4], [18.9], [22.3], [18.3], [12.4], [8.8], [11.0], [17.0], [8.7], [6.9], [14.2], [5.3], [11.0], [11.8], [17.3], [11.3], [13.6], [21.7], [20.2], [12.0], [16.0], [12.9], [16.7], [14.0], [7.3], [19.4], [22.2], [11.5], [16.9], [16.7], [20.5], [25.4], [17.2], [16.7], [23.8], [19.8], [19.7], [20.7], [15.0], [7.2], [12.0], [5.3], [19.8], [18.4], [21.8], [17.1], [20.9], [14.6], [12.6], [12.2], [9.4], [15.9], [6.6], [15.5], [7.0], [16.6], [15.2], [19.7], [10.6], [6.6], [11.9], [24

In [23]:
X = torch.tensor(x_train[:60], dtype=torch.float)  # 60 X 3 tensor
y = torch.tensor(y_train[:60], dtype=torch.float)  # 60 X 1 tensor

# scale units by max value
X_max, _ = torch.max(X, 0)
X = torch.div(X, X_max)
y = y / 100  # for max test score is 100

Dữ liệu dùng để test

In [24]:
# sample input x for predicting
x_predict = torch.tensor(([38.2, 3.7, 13.8]), dtype=torch.float)  # 3 X 1 tensor
# correct one is 7.6

# scale input x by max value
x_predict_max, _ = torch.max(x_predict, 0)
x_predict = torch.div(x_predict, x_predict_max)

Chạy model

In [25]:
NN = FFNN()

epochs = 1000 # trains the NN 1,000 times
for i in range(epochs):
    # print mean sum squared loss
    print("#" + str(i) + " Loss: " + str(torch.mean(((y - NN(X)) ** 2)).detach().item()))

    # training with learning rate = 0.1
    NN.train(X, y, 0.1)

# save weights
SaveModel(NN, "FFNN")

# load saved weights
result = LoadModel("FFNN")

# predict x input
result.predict(x_predict)

#0 Loss: 0.022732950747013092
#1 Loss: 0.022660115733742714
#2 Loss: 0.02258482575416565
#3 Loss: 0.022506969049572945
#4 Loss: 0.022426428273320198
#5 Loss: 0.022343087941408157
#6 Loss: 0.022256817668676376
#7 Loss: 0.02216748334467411
#8 Loss: 0.022074948996305466
#9 Loss: 0.021979061886668205
#10 Loss: 0.021879667416214943
#11 Loss: 0.021776605397462845
#12 Loss: 0.021669697016477585
#13 Loss: 0.021558769047260284
#14 Loss: 0.021443627774715424
#15 Loss: 0.021324072033166885
#16 Loss: 0.021199895069003105
#17 Loss: 0.021070880815386772
#18 Loss: 0.020936794579029083
#19 Loss: 0.020797397941350937
#20 Loss: 0.020652439445257187
#21 Loss: 0.020501647144556046
#22 Loss: 0.020344754680991173
#23 Loss: 0.02018147148191929
#24 Loss: 0.020011497661471367
#25 Loss: 0.019834520295262337
#26 Loss: 0.01965022087097168
#27 Loss: 0.019458262249827385
#28 Loss: 0.019258299842476845
#29 Loss: 0.019049979746341705
#30 Loss: 0.018832935020327568
#31 Loss: 0.018606793135404587
#32 Loss: 0.0183711759